In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from PIL import ImageGrab
import time

import tkinter as tk
from tkinter import Message, Text
import cv2
import shutil
import csv
import numpy as np
from PIL import Image, ImageTk
import pandas as pd
import datetime
import tkinter.ttk as ttk
import tkinter.font as font
from pathlib import Path

In [2]:
def labeling():
    path = 'ImagesAttendance'
    images = []
    classNames = []
    myList = os.listdir(path)
    #print(myList)
    for cl in myList:
        curImg = cv2.imread(f'{path}/{cl}')
        images.append(curImg)
        classNames.append(os.path.splitext(cl)[0])
    #print(classNames)
    return [images,classNames]

In [3]:
 def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

In [4]:
def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

In [5]:
def writecsv(name,starttime,endtime,t,p):
    percent = (p/t)*100
    if percent>=70:
        s = "Present"
    else:
        s = "Absent"
    with open("attend.csv","a") as f:
        f.writelines(f'\n{name},{starttime},{endtime},{percent},{s}\n')
            

In [6]:
import time
def note(): 
    images,classNames = labeling()
    encodeListKnown = findEncodings(images)
    print('Encoding Complete')

    cap = cv2.VideoCapture(0)

    t=0
    p=0
    
    #now = datetime.now()
    t1 = time.localtime()
    st = time.strftime("%H:%M:%S",t1)
    while True: 
        start=time.time()
        f=0
        success, img = cap.read()

        imgS = cv2.resize(img,(0,0),None,0.25,0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

        facesCurFrame = face_recognition.face_locations(imgS)
        encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)

        for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)

            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                f=1
                name = classNames[matchIndex].upper()

                y1,x2,y2,x1 = faceLoc
                y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
                cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
                cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                #markAttendance(name)

        cv2.imshow('Webcam',img)
        key=cv2.waitKey(1)
        if(key==ord("q")):
            break
        iter_time = time.time()-start
        t+=iter_time
        if f==1:
            p+=iter_time
    t1 = time.localtime()
    et = time.strftime("%H:%M:%S",t1)
    
    writecsv(name,st,et,t,p)
    
    print("{:.2f} - {:.2f}".format(t,p))
    cap.release()
    cv2.destroyAllWindows()

In [7]:


window = tk.Tk()
window.title("Face_Recogniser")
window.configure(background ='white')
window.grid_rowconfigure(0, weight = 1)
window.grid_columnconfigure(0, weight = 1)
message = tk.Label(
    window, text ="Face-Recognition-System",
    bg ="blue", fg = "white", width = 50,
    height = 3, font = ('times', 30, 'bold'))
    
message.place(x = 200, y = 20)

lbl = tk.Label(window, text = "No.",
width = 20, height = 2, fg ="blue",
bg = "white", font = ('times', 15, ' bold ') )
lbl.place(x = 400, y = 200)

txt = tk.Entry(window,
width = 20, bg ="white",
fg ="green", font = ('times', 15, ' bold '))
txt.place(x = 700, y = 215)

lbl2 = tk.Label(window, text ="Name",
width = 20, fg ="blue", bg ="white",
height = 2, font =('times', 15, ' bold '))
lbl2.place(x = 400, y = 300)

txt2 = tk.Entry(window, width = 20,
bg ="white", fg ="green",
font = ('times', 15, ' bold ') )
txt2.place(x = 700, y = 315)

def Capture(name):
    video=cv2.VideoCapture(0)
    
    os.makedirs(".//ImagesAttendance",exist_ok=True)
    #name = input("Enter Your Name : ")
    
    a=1
    while True:
        a=a+1
        check,image=video.read()

        cv2.imshow("Press 'q' To Capture", image)
    
        key=cv2.waitKey(1)
        if(key==ord("q")):
            break
    
    cv2.imwrite("ImagesAttendance//"+name+".jpg",image)
    
    time.sleep(3)
    video.release()
    cv2.destroyAllWindows()



takeImg = tk.Button(window, text ="Capture",
command =lambda: Capture(txt2.get()), fg ="white", bg ="blue",
width = 20, height = 3, activebackground = "Red",
font =('times', 15, ' bold '))
takeImg.place(x = 200, y = 500)

# trainImg = tk.Button(window, text ="Training",
# command = note, fg ="white", bg ="green",
# width = 20, height = 3, activebackground = "Red",
# font =('times', 15, ' bold '))
# trainImg.place(x = 500, y = 500)

trackImg = tk.Button(window, text ="Start",
command = note, fg ="white", bg ="blue",
width = 20, height = 3, activebackground = "Red",
font =('times', 15, ' bold '))
trackImg.place(x = 650, y = 500)

quitWindow = tk.Button(window, text ="End",
command = window.destroy, fg ="white", bg ="blue",
width = 20, height = 3, activebackground = "Red",
font =('times', 15, ' bold '))
quitWindow.place(x = 1100, y = 500)


window.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\badri\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-7-c739740ced14>", line 59, in <lambda>
    command =lambda: Capture(txt2.get()), fg ="white", bg ="blue",
  File "<ipython-input-7-c739740ced14>", line 44, in Capture
    cv2.imshow("Press 'q' To Capture", image)
cv2.error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-k1ohfcms\opencv\modules\highgui\src\window.cpp:404: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'



Encoding Complete
16.14 - 16.14


In [43]:
# Import required Libraries
from tkinter import *
from PIL import Image, ImageTk
import cv2

# Create an instance of TKinter Window or frame
win = Tk()

# Set the size of the window
win.geometry("700x350")

# Create a Label to capture the Video frames
label =Label(win)
label.grid(row=0, column=0)
cap= cv2.VideoCapture(0)

# Define function to show frame
def show_frames():
   # Get the latest frame and convert into Image
   cv2image= cv2.cvtColor(cap.read()[1],cv2.COLOR_BGR2RGB)
   img = Image.fromarray(cv2image)
   # Convert image to PhotoImage
   imgtk = ImageTk.PhotoImage(image = img)
   label.imgtk = imgtk
   label.configure(image=imgtk)
   # Repeat after an interval to capture continiously
   label.after(20, show_frames)

show_frames()
win.mainloop()